In this project, we'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/). We'll use SQL in Jupyter Notebook to explore and analyze data from [this](https://www.cia.gov/library/publications/download) database. The database we 'll be working with that were collected in 2015. 

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## Explore the Data

In [2]:
%%sql
SELECT*
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


[(1, 'af', 'Afghanistan', 652230, 652230, 0, 32564342, 2.32, 38.57, 13.89, 1.51, '2015-11-01 13:19:49.461734', '2015-11-01 13:19:49.461734'),
 (2, 'al', 'Albania', 28748, 27398, 1350, 3029278, 0.3, 12.92, 6.58, 3.3, '2015-11-01 13:19:54.431082', '2015-11-01 13:19:54.431082'),
 (3, 'ag', 'Algeria', 2381741, 2381741, 0, 39542166, 1.84, 23.67, 4.31, 0.92, '2015-11-01 13:19:59.961286', '2015-11-01 13:19:59.961286'),
 (4, 'an', 'Andorra', 468, 468, 0, 85580, 0.12, 8.13, 6.96, 0.0, '2015-11-01 13:20:03.659945', '2015-11-01 13:20:03.659945'),
 (5, 'ao', 'Angola', 1246700, 1246700, 0, 19625353, 2.78, 38.78, 11.49, 0.46, '2015-11-01 13:20:08.625072', '2015-11-01 13:20:08.625072')]

Here is the short discription for each columns' name:
- *id*: identification number.
- *code*: abreviation for country's name. 
- *name*: the name of the country.
- *area*: the total land and sea area of the country.
- *area_land*: the country's land area in square kilometers.
- *area_water*: the country's waterarea in square kilometers.
- *population*: the country's population.
- *population_growth*: the country's population growth as percentage.
- *birth_date*: the country's birth rate, or the number of births a year per 1,000 people.
- *death_rate*: the country's death rate, or the number of births a year per 1,000 people. 
- *migration_rate*: migration rate.
- *created_at*: time date of first created data.
- *update_at*: time date of last updated. 

Let's start by calculating some summary statistics and see what they tell us.

## Summary Statistics

In [3]:
%%sql
SELECT 
MIN(population) AS min_pop, 
MAX(population) AS max_pop,
MIN(population_growth) AS min_pop_growth,
MAX(population_growth) AS max_pop_growth
FROM facts;

 * sqlite:///factbook.db
Done.


[(0, 7256490011, 0.0, 4.02)]

Findings:
- There's a country with a population of 0.
- There's a country with a population of 7,256,490,011 (around 7.3 billion residents)


## Explore Outliers

We'll then use subqueries to zoom in on just these countries without using the specific values.

In [4]:
%%sql
SELECT * FROM facts
WHERE population == (SELECT MIN(population) FROM facts);

 * sqlite:///factbook.db
Done.


[(250, 'ay', 'Antarctica', None, 280000, None, 0, None, None, None, None, '2015-11-01 13:38:44.885746', '2015-11-01 13:38:44.885746')]

Oh so it's Antarctica that has 0 population. This seems to match the CIA Factbook [page for Antarctica](https://www.cia.gov/library/publications/the-world-factbook/geos/ay.html).


In [5]:
%%sql
SELECT * FROM facts
WHERE population == (SELECT MAX(population) FROM facts);

 * sqlite:///factbook.db
Done.


[(261, 'xx', 'World', None, None, None, 7256490011, 1.08, 18.6, 7.8, None, '2015-11-01 13:39:09.910721', '2015-11-01 13:39:09.910721')]

It's actually the whole world population that has more then 7.2 billion residents. Now after knowing this, we should take a look one more time with the summary statistics we calculated earlier, while excluding the row for the whole world. 

## Summary Statistics Revisited

In [6]:
%%sql
SELECT 
MIN(population) AS min_pop,
MAX(population) AS max_pop,
MIN(population_growth) AS min_pop_growth,
MAX(population_growth) AS max_pop_growth
FROM facts
WHERE name <> 'World';

 * sqlite:///factbook.db
Done.


[(0, 1367485388, 0.0, 4.02)]

Now the country has the world biggest population of 1.4 billion people. 

## Explore Average Population and Area

Let's explore density. Density depends on the population and the country's area. Let's look at the average values for these 2 columns.

In [7]:
%%sql
SELECT AVG(population) AS avg_population,
AVG(area) AS avg_area
FROM facts
WHERE name <> 'World'

 * sqlite:///factbook.db
Done.


[(32242666.56846473, 555093.546184739)]

We can see that the average population is around 32.2 million and the average area is 555 thousand square kilometers. 

## Find Densely Populated Countries

We'll find countries that are densely populated. We'll identify countries that have:
- Above average values for population.
- Below average values for area.

In [8]:
%%sql
SELECT * FROM facts
WHERE population > (SELECT AVG(population) FROM facts)
AND area < (SELECT AVG(area) FROM facts);

 * sqlite:///factbook.db
Done.


[(14, 'bg', 'Bangladesh', 148460, 130170, 18290, 168957745, 1.6, 21.14, 5.61, 0.46, '2015-11-01 13:20:52.753843', '2015-11-01 13:20:52.753843'),
 (65, 'gm', 'Germany', 357022, 348672, 8350, 80854408, 0.17, 8.47, 11.42, 1.24, '2015-11-01 13:25:21.942190', '2015-11-01 13:25:21.942190'),
 (85, 'ja', 'Japan', 377915, 364485, 13430, 126919659, 0.16, 7.93, 9.51, 0.0, '2015-11-01 13:27:08.040081', '2015-11-01 13:27:08.040081'),
 (138, 'rp', 'Philippines', 300000, 298170, 1830, 100998376, 1.61, 24.27, 6.11, 2.09, '2015-11-01 13:31:23.643550', '2015-11-01 13:31:23.643550'),
 (173, 'th', 'Thailand', 513120, 510890, 2230, 67976405, 0.34, 11.19, 7.8, 0.0, '2015-11-01 13:34:11.057976', '2015-11-01 13:34:11.057976'),
 (185, 'uk', 'United Kingdom', 243610, 241930, 1680, 64088222, 0.54, 12.17, 9.35, 2.54, '2015-11-01 13:35:09.362933', '2015-11-01 13:35:09.362933'),
 (192, 'vm', 'Vietnam', 331210, 310070, 21140, 94348835, 0.97, 15.96, 5.93, 0.3, '2015-11-01 13:35:42.896553', '2015-11-01 13:35:42.896553')]

Some of these countries are generally known to be densely populated, so we have confidence in our results. 